In [ ]:
#import library and all the required key
!pip install sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import tweepy
#import csv, re
import re,string
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import itertools
import collections

class data_analisis:
  def __init__ (self):
    pass
  
  def open_file(self, file):
      con = sqlite3.connect(file)
      df = pd.read_sql_query("SELECT * from data_tweet", con)
      con.close()
      print (df)
      return df
    
  def praproses(self, data):
      clean_tweets = []
      tweets = []
      # data['Tweets'] = tweets
      # tweet = data["Tweets"]
      for tweet in data['Tweets']:
        def hapus_tanda(tweet): 
            tanda_baca = set(string.punctuation)
            tweet = ''.join(ch for ch in tweet if ch not in tanda_baca)
            return tweet
            
        def hapus_katadouble(s): 
            pattern = re.compile(r"(.)\1{1,}", re.DOTALL)
            return pattern.sub(r"\1\1", s)
        
        #mengecilkan huruf
        tweet=tweet.lower()
        #hapus link
        tweet = re.sub(r'\\u\w\w\w\w', '', tweet)
        tweet=re.sub(r'http\S+','',tweet)
        #hapus @username
        tweet=re.sub('@[^\s]+','',tweet)
        #hapus #tagger 
        tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
        #hapus tanda baca
        tweet=hapus_tanda(tweet)
        #hapus angka yang berada dalam string 
        tweet=re.sub(r'\w*\d\w*', '',tweet).strip()
        #hapus repetisi karakter 
        tweet=hapus_katadouble(tweet)
        #stemming
        factory = StemmerFactory()
        stemmer = factory.create_stemmer()
        tweet = stemmer.stem(tweet)
        clean_tweets.append(tweet)
        
      # data['clean'] = clean_tweets
      return(clean_tweets)

  def hitung_kata(self, data):

    words_in_tweet = [tweet.lower().split() for tweet in data]
    
    # words_in_tweet[:2]
    data = data[0].lower().split()
    print (data)
    # List of all words across tweets
    all_words_no_urls = list(itertools.chain(*words_in_tweet))

    # Create counter
    counts_no_urls = collections.Counter(data)
    counts_no_urls.most_common(15)

    clean_tweets = pd.DataFrame(counts_no_urls.most_common(15),  columns=['words', 'count'])
    print (clean_tweets)


  def analisis_sentiment(self, clean_data):
    pos_list= open("./kata_positif.txt","r")
    pos_kata = pos_list.readlines()
    neg_list= open("./kata_negatif.txt","r")
    neg_kata =  neg_list.readlines()

    Sentiment = []
    for item in clean_data:
      count_p = 0
      count_n = 0
      for kata_pos in pos_kata:
          if kata_pos.strip() in item:
            count_p +=1
      for kata_neg in neg_kata:
          if kata_neg.strip() in item:
            count_n +=1

      Sentiment.append(count_p - count_n)
    
    return (Sentiment)

  def visualisasi(self, Sentiment):
    #Menghitung mean, median, dan standart deviasi
    print ("Nilai rata-rata: "+str(np.mean(Sentiment)))
    print ("Median: "+str(np.median(Sentiment)))
    print ("Standart Deviasi: "+str(np.std(Sentiment)))

    labels, counts = np.unique(Sentiment, return_counts=True)
    plt.bar(labels, counts, align='center')
    plt.gca().set_xticks(labels)
    plt.show()

file = "tugasakhir.db"
app = data_analisis()
#open the database file
data = app.open_file(file)



                      ID  ...                    Lokasi
0    1319426925522161664  ...                          
1    1319424690234904577  ...                 Indonesia
2    1319422818178584578  ...                          
3    1319421172656730112  ...   Tasikmalaya, Jawa Barat
4    1319419774221258753  ...                 Indonesia
..                   ...  ...                       ...
747  1318733744459579392  ...                          
748  1318730579324514305  ...                          
749  1318719833559629825  ...                 Indonesia
750  1318719672422858752  ...                     bogor
751  1318714322860662784  ...             di bumi ALLAH

[752 rows x 5 columns]


In [ ]:
data.columns

Index(['ID', 'Tweets', 'Waktu', 'Akun', 'Lokasi'], dtype='object')

In [ ]:
data.groupby('Lokasi')['Lokasi'].count().max

<bound method Series.max of Lokasi
                            330
 Indonesia                    2
 Tasikmalaya, Jawa Barat      1
+62                           3
.......                       1
                           ... 
🇮🇩 Indonesia                  1
🇮🇩#NetizenProNKRI🇮🇩           1
🌍                             1
🕋Atjeh 🕌●๋•                   1
🤡                             2
Name: Lokasi, Length: 273, dtype: int64>